In [1]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2,preprocess_input
from tensorflow import keras
from tensorflow.keras.layers import *
from tensorflow import keras
import numpy as np
import os
from tensorflow.keras.preprocessing import image
from skimage import io, color,transform
import pickle


In [2]:
X=[]
for img in os.listdir('C:/Users/HP/coloi3/images/Train1'):
    X.append(image.img_to_array(image.load_img('C:/Users/HP/coloi3/images/Train1/'+img,target_size=(256,256))))
    
X = np.array(X,dtype=float)
Xtrain = 1.0/255*X[:int(0.97*len(X))]
Xtest = 1.0/255*X[int(0.97*len(X)):]


    
  #Globlal Feature extractor
inception = InceptionResNetV2(weights='imagenet',include_top=True)
# inception.load_weights('inception_resnet_v2_weights_tf_dim_ordering_tf_kernels.h5')
# print(inception.layers)  

In [3]:

graph = tf.get_default_graph()

embed_inp = Input(shape=(1000,))
# inception.graph = tf.get_default_graph() #Makes inception model graph == default graph

#Encoder Network via functional API
enc_inp = Input(shape=(256,256,1))
encode = Conv2D(64,(3,3),strides=2,activation='relu',padding='same',name='d')(enc_inp)
encode = Conv2D(128,(3,3),strides=1,activation='relu',padding='same',name='e')(encode)
encode = Conv2D(128,(3,3),strides=2,activation='relu',padding='same',name='f')(encode)
encode = Conv2D(256,(3,3),strides=1,activation='relu',padding='same',name='g')(encode)
encode = Conv2D(256,(3,3),strides=2,activation='relu',padding='same',name='h')(encode)
encode = Conv2D(512,(3,3),strides=1,activation='relu',padding='same',name='i')(encode)
encode = Conv2D(512,(3,3),strides=1,activation='relu',padding='same',name='j')(encode)
encode = Conv2D(256,(3,3),strides=1,activation='relu',padding='same',name='k')(encode)

#fusion layer
fuse = RepeatVector(32*32)(embed_inp)
fuse = Reshape([32,32,1000])(fuse)
fuse = concatenate([encode,fuse],axis=3)
fuse = Conv2D(256,(1,1),activation='relu',padding='same',name='l')(fuse)


#Decoder for final colorization
decode = Conv2D(128,(3,3),activation='relu',padding='same',name='m')(fuse)
decode = UpSampling2D(size=(2,2),name='A')(decode)
decode = Conv2D(64,(3,3),activation='relu',padding='same',name='n')(decode)
decode = UpSampling2D(size=(2,2),name='B')(decode)
decode = Conv2D(32,(3,3),activation='relu',padding='same',name='o')(decode)
decode = Conv2D(16,(3,3),activation='relu',padding='same',name='p')(decode)
decode = Conv2D(2,(3,3),activation='tanh',padding='same',name='q')(decode)
decode = UpSampling2D(size=(2,2),name='C')(decode)

#Finally,model

model = Model(inputs=[enc_inp,embed_inp],outputs=decode)


In [4]:
#Embedding for global feature extractor(I mean InceptioResnetV2)
def emb_for_gfe(rgb):
    rgb_resized = []
#     print(rgb.shape)
    for i in rgb:
        i = transform.resize(i,(299,299,3),mode='constant')
#         i = i.reshape((299,299,3))
        rgb_resized.append(i)
    rgb_resized = np.array(rgb_resized)
    rgb_resized = preprocess_input(rgb_resized)
   

    with graph.as_default():
        embed = inception.predict(rgb_resized)
    return embed

#Transorms
datagen = image.ImageDataGenerator( shear_range=0.4,
                                    zoom_range=0.4,
                                    rotation_range=40,
                                    horizontal_flip=True)
b_size = 3


def generator(b_size):
    for batch in datagen.flow(Xtrain,batch_size=b_size):
        gray_rgb = color.gray2rgb(color.rgb2gray(batch))
        embed = emb_for_gfe(gray_rgb)
        lab = color.rgb2lab(batch)
        x_batch = (lab[:,:,:,0])#Luminence
        x_batch = x_batch.reshape(x_batch.shape + (1,))
        y_batch = lab[:,:,:,1:]/128
        yield([x_batch,embed],y_batch)
        
        
        
    

In [7]:
#Train
with graph.as_default():

    model.compile(optimizer='adam', loss='mse')
    model.fit_generator(generator(b_size),epochs=4,steps_per_epoch=1)


#test
test_rgb = color.gray2rgb(color.rgb2gray(Xtest))
embed_test = emb_for_gfe(test_rgb)
lab = color.rgb2lab(Xtest)
x_test = (lab[:,:,:,0])#Luminence
x_test = x_test.reshape(x_test.shape + (1,))
y_test = lab[:,:,:,1:]/128

print(model.evaluate([x_test,embed_test], y_test,batch_size=1))

Epoch 1/4


C:\Users\HP\Anaconda3\lib\site-packages\skimage\transform\_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


1/1 [==============================] - 11s 11s/step - loss: 0.0171
Epoch 2/4
1/1 [==============================] - 1s 857ms/step - loss: 0.2852
Epoch 3/4
1/1 [==============================] - 1s 860ms/step - loss: 0.0074
Epoch 4/4
1/1 [==============================] - 2s 2s/step
0.0032526645809412003
